In [ ]:
# Imports
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
from PIL import Image
import datetime
import random
import tensorflow as tf
from pathlib import Path

%load_ext tensorboard
!rm -rf ./logs/


path_SM = '/content/gdrive/My Drive/Archana/Colab_Notebooks/Data/SMAP_SM_global/'
path_PPT = '/content/gdrive/My Drive/Archana/Colab_Notebooks/Data/SMAP_PPT_global/'

for i in range(1,32):


  start = datetime.datetime(2017,12,i,1,30,0) # START DATETIME
  end = datetime.datetime(2017,12,i,22,30,0) # END DATETIME
  delta = datetime.timedelta(hours=3) # DELTA T. For SMAP L4 DELTA=3hours

  frames_SM = []
  frames_PPT = []

  while start <= end:
    d = start.strftime("%Y%m%d%H%M%S")

    img_SM = Image.open(path_SM+d+"_sm.jpeg")
    img_PPT = Image.open(path_PPT+d+"_ppt.jpeg")

    img_SM.show()
    img_PPT.show()

    # print(np.shape(img_SM))
    # print(np.shape(img_PPT))

    f_SM = img_SM.convert(mode='RGB')
    f_PPT = img_PPT.convert(mode='RGB')

    f_SM = np.array(f_SM)
    f_PPT = np.array(f_PPT)

    f_SM = np.sum(f_SM, axis=2)
    f_PPT = np.sum(f_PPT, axis=2)

    f_SM = np.array(f_SM)
    f_PPT = np.array(f_PPT)

    f_SM = f_SM.reshape(f_SM.shape[0], f_SM.shape[1], 1)
    f_PPT = f_PPT.reshape(f_PPT.shape[0], f_PPT.shape[1], 1)

    frames_SM.append(f_SM)
    frames_PPT.append(f_PPT)

    start += delta

  frames_SM = np.array(frames_SM)
  frames_PPT = np.array(frames_PPT)

  print(np.shape(frames_SM))
  print(np.shape(frames_PPT))

  window = 3 # window is the interval of prediction. 
  # Eg. window = 3 ==> three inputs, three outputs. Using 6hr inputs(3 i/ps) the next 6hr(3 o/ps) is predicted

  # Train data

  train_frames_SM = frames_SM[0:(len(frames_SM)),:,:,:]
  train_frames_PPT = frames_PPT[0:(len(frames_PPT)),:,:,:]

  print(f"Shape of train_frames_SM is {np.shape(train_frames_SM)}")
  print(f"Shape of train_frames_PPT is {np.shape(train_frames_PPT)}")

  # Normalization

  maxi_SM = np.max(train_frames_SM)
  mini_SM = np.min(train_frames_SM)
  train_frames_SM = (train_frames_SM-mini_SM)/(maxi_SM-mini_SM)

  maxi_PPT = np.max(train_frames_PPT)
  mini_PPT = np.min(train_frames_PPT)
  train_frames_PPT = (train_frames_PPT-mini_PPT)/(maxi_PPT-mini_PPT)

  X_train_SM, X_train_PPT, y_train = [], [], []

  for i in range(0, len(train_frames_SM)-2*window, 1):
    X_train_SM.append(train_frames_SM[i:i+window,::,::,::])
    X_train_PPT.append(train_frames_PPT[i:i+window,::,::,::])
    y_train.append(train_frames_SM[(i+window):(i+window+window),::,::,::])

  X_train_SM = np.array(X_train_SM)
  X_train_PPT = np.array(X_train_PPT)  
  y_train = np.array(y_train) 

  print(f"Shape of X_train_SM is {np.shape(X_train_SM)}")
  print(f"Shape of X_train_PPT is {np.shape(X_train_PPT)}")
  print(f"Shape of y_train is {np.shape(y_train)}")

  # Validation data

  r=random.sample(range(len(train_frames_SM)-2*window), 1)

  X_val_SM = X_train_SM[r, ::, ::, ::, ::]
  X_val_PPT = X_train_PPT[r, ::, ::, ::, ::]

  y_val = y_train[r, ::, ::, ::, ::]

  X_train_SM = np.delete(X_train_SM, (r), axis=0)
  X_train_PPT = np.delete(X_train_PPT, (r), axis=0)

  y_train = np.delete(y_train, (r), axis=0) 

  # ConvLSTM model for SM
  inp_SM = tf.keras.layers.Input((window, 1624, 3856, 1))
  x1   =  tf.keras.layers.ConvLSTM2D(filters=2, kernel_size=(3, 3), padding='same', return_sequences=True) (inp_SM)
  x1   =  tf.keras.layers.ConvLSTM2D(filters=2, kernel_size=(3, 3), padding='same', return_sequences=True )(x1)
  m1  =  tf.keras.models.Model(inp_SM, x1)

  # ConvLSTM model for PPT
  inp_PPT = tf.keras.layers.Input((window, 1624, 3856, 1))
  x2   =  tf.keras.layers.ConvLSTM2D(filters=2, kernel_size=(3, 3), padding='same', return_sequences=True) (inp_PPT)
  x2   =  tf.keras.layers.ConvLSTM2D(filters=2, kernel_size=(3, 3), padding='same', return_sequences=True)(x2)
  m2  =  tf.keras.models.Model(inp_PPT, x2)

  m1.summary()
  m2.summary()

  # Combined model
  combinedInput = tf.keras.layers.concatenate([m1.output, m2.output])

  x = tf.keras.layers.ConvLSTM2D(filters=2, kernel_size=(3, 3), padding='same', return_sequences=True)(combinedInput)
  x = tf.keras.layers.Dropout(0.2)(x, training=True)
  x = tf.keras.layers.Conv3D(filters=1, kernel_size=(3, 3, 3), activation='sigmoid', padding='same')(x)

  model = tf.keras.models.Model(inputs=[m1.input, m2.input], outputs=x)

  model.summary()

  if Path('/content/gdrive/My Drive/Archana/Colab_Notebooks/Models/global_model_v1.h5').is_file():
    print("Partially trained model found! Retraining an old model")
    train_model = tf.keras.models.load_model('/content/gdrive/My Drive/Archana/Colab_Notebooks/Models/global_model_v1.h5')
  else:
    print("New model is being trained")
    train_model = model


  best_model_file = 'vgg.h5'
  best_model = tf.keras.callbacks.ModelCheckpoint(best_model_file, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
  train_model.compile(optimizer='adam', loss="mse")

  log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

  my_callbacks = [best_model, tensorboard_callback]

  history = train_model.fit(x=[X_train_SM, X_train_PPT], y=y_train, validation_data=([X_val_SM, X_val_PPT], y_val), epochs=5, callbacks=my_callbacks,batch_size=20)
  train_model.load_weights(best_model_file)
  tf.keras.models.save_model(train_model,'/content/gdrive/My Drive/Archana/Colab_Notebooks/Models/global_model_v1.h5')
